# 9.1 Playing with ROS

In this chapter we are going to play a bit with different robotic techniques in the context of the [Robot Operating System (ROS) ecosystem](https://www.ros.org/).

For that, we are going to use the [Stage simulator](http://playerstage.sourceforge.net/doc/Stage-3.2.1/index.html): 

> Stage is a robot simulator, it provides a virtual world populated by mobile robots and sensors, along with various objects for the robots to sense and manipulate. ROS provides a node, called [stage_ros](http://wiki.ros.org/stage_ros ), which wraps the core functionality of Stage, like the utilization of 2D laser scanners or robotic bases yielding odometry information.

<center>
<img src="./images/stage_ros_official_example.png" width=600/> $\\[5px]$
<figcaption>Fig 1. Example of a simulation with Stage showing several robots moving around an environment.</figcaption>
</center>

## 9.1.1 Getting ready: the provided Virtual Machine and the pre-installed ros-pkgs

For those not willing to install and configure ROS, a virtual machine (Virtual Box) is provided together with this notebook holding a Ubuntu 18.04 OS. It comes with ROS Melodic installed, and a catkin workspace ready to use.

Some indications:
- User of the OS: robotics, password: robotics.
- ROS installation path: /opt/ros/melodic
- Catkin workspace path: /home/robotics/catkin_ws

Note-> If you want to install ROS in your computer, you can follow the detailed guide in: http://wiki.ros.org/melodic/Installation, and install the Desktop-Full version of ROS. If you choose this option, you have to know that the `open_gmapping` package is not available for ROS Melodic in the Ubuntu package repository. That package is a wrapper for **GMapping**, a SLAM method **based on particle filters** that we are going to use. However, it is easy to compile it from source. For doing that, execute the following commands:

```
cd ~/catkin_ws/src
git clone https://github.com/ros-perception/openslam_gmapping src/openslam_gmapping
git clone https://github.com/ros-perception/slam_gmapping src/slam_gmapping
cd ..
Catkin_make
```

You would also need to install the other packages that are already provided with the virtual machine. You can find them in the *Your first robotic explorer pkgs* file in the virtual campus. To install them you just have to copy its content in your `~/catkin_ws/src` directory and compile them with `catkin_make`.

## 9.1.2 Insight into Stage

Stage comes also pre-installed into the virtual machine. Stage simulates a world as defined in a `.world` file. This file tells stage everything about the world, from obstacles (usually represented via a bitmap to be used as a kind of background), to robots and other objects in the virtual environment. The node `stage_ros` only exposes a subset of Stage's functionality via ROS. Specifically, it finds the Stage models of type laser, camera (rgbd) and position, and maps these models to the ROS topics:

- laser_scan (`sensor_msgs/LaserScan`)
- image (`sensor_msgs/Image`)
- depth (`sensor_msgs/Image`)
- odom (`nav_msgs/Odometry`)
- base_pose_ground_truth (`nav_msgs/Odometry`) *NOT USE!!!*

You can run this node by executing the following command:

```
    rosrun stage_ros stageros $(rospack find missions_pkg)/world/robotics-house4.world
```

Note→ Remember that the ROS-MASTER must always be launched before any other node (executing `roscore`)


You will see an interface like the one shown below, where the robot is represented as a blue box, and it is equipped with a 2D laser scanner. Walls are represented as gray, tall rectangles, which define the rooms within the environment where the robot is going to operate. Spend some minutes taking a look at the `View` options within the Stage interface, since they can be useful for checking different types of information. $\\[20px]$

<center>
<img src="./images/stage_example_1.png" width=600/> $\\[1px]$
<figcaption>Fig 2. Example of a robot (blue square) and the readings from the laser scanner (green).</figcaption>
</center>

## 9.1.3 Some basic movement commands

You can open a new terminal and command the robot directly by publishing in the `cmd_vel` topic:

```
rostopic pub /cmd_vel geometry_msgs/Twist '[-0.5, 0, 0]' '[0, 0, 0]' -r 100
```

Since this is a very tedious form of controlling a robot, you can make use of the community and employ different packages that control a robot through the keyboard. For example, you can use the `keyboard_control` pkg that is also provided with this notebook.

You can launch this simple example with the following command, which makes use of [`roslaunch`](http://wiki.ros.org/roslaunch), a tool for easily launching multiple ROS nodes, as well as setting parameters on the Parameter Server: 

```
roslaunch missions_pkg demo_1_stage_keyboard.launch
```

Take care commanding the robot, it can crash otherwise! $\\[20px]$

<center>
<img src="./images/stage_example_2.png" width=600/> $\\[1px]$
<figcaption>Fig 3. Example of a sad robot.</figcaption>
</center>

### **<span style="color:green"><b><i>ASSIGNMENT 1: Manually controlling the robot</i></b></span>** 

**What to do?**

- Command the robot using the *launch file* provided above,
- activate the needed options for visualizing the path followed by the robot in Stage, and
- take a picture of such simulation and provide it below.

<p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>

# 9.1.4 SLAM demo

Although we will use Stage as a robotic simulator, it is a good practice to visualize the data coming from the robot outside the simulator. That way, when we use a real robot, the interface will be the same. [RVIZ](http://wiki.ros.org/rviz) is a 3D visualization tool for ROS with a lot of pre-configured ROS message types.

As a demo for introducing ROS and RVIZ, let’s take a look at a very employed package for SLAM: **Gmapping**. This package contains a ROS wrapper for **OpenSlam's Gmapping**. The `gmapping` package provides laser-based SLAM (Simultaneous Localization and Mapping) as a ROS node called `slam_gmapping`. Using `slam_gmapping`, you can create a 2-D occupancy grid map (like a building floorplan) from laser and pose data collected by a mobile robot.

As a simple exercise, you can load the Stage simulator, then launch the `gmapping` node, and finally move manually the robot to see how the map builds up while localizing the robot (SLAM). For example, you can launch this demo with the command:

```
roslaunch missions_pkg demo_2_slam.launch
```

$\\[10px]$

<center>
<img src="./images/rviz_example.png"/> $\\[1px]$
<figcaption>Fig 4. Example of a map under construction.</figcaption>
</center>

### **<span style="color:green"><b><i>ASSIGNMENT 2: Building maps</i></b></span>** 

Launch the previous *launch file* in two of the provided environments of your choice. Said environments are defined in the *world files*:

- `robotics-house1.world`
- `robotics-house2.world`
- `robotics-house3.world`
- `robotics-house4.world`
- `robotics-offices1.world`

The content of the `demo_2_slam.launch` file is as follows:

```
<!-- Launch file for the Robotics exercices with ROS -->

<launch>    

    ### ROBOT SIMULATION ###
    <param name="use_sim_time" value="true" />
    <include file="$(find missions_pkg)/launch/simbot_stage.launch" >
        <arg name="world_file" value="-d $(find missions_pkg)/world/robotics-house4.world" />
    </include>
    
    ### URDF model "giraff" robot ###
    <include file="$(find missions_pkg)/launch/simbot_urdf.launch" />
       
    ### NAVIGATION ###
    <include file="$(find missions_pkg)/launch/simbot_keyboard_control.launch" />
    
    ### MAPPING/SLAM ###
    <include file="$(find missions_pkg)/launch/simbot_gmapping.launch" />
  
    ### RVIZ  ###
    <node name="rviz" pkg="rviz" type="rviz" respawn="false" output="screen" args="-d $(find missions_pkg)/rviz/simbot_mapcreation.rviz"/>
   
</launch>

```

So you have to modify the *world file* in the `world_file` parameter for launching the SLAM demo with each environment.

**What to provide?** 

Include here an image of each resultant map.

  <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>

# 9.1.5 The robotic explorer

At this point in the notebook we will follow an scheme a bit different from the previous ones, in the sense that we are going to develop an algorithm that has not been explicitly covered in a lecture. Concretely, after being using the SLAM solution that we developed in previous notebooks, the managers at Nirvana are pleased with it, but the building of the map requires to manually specify the path that the robot must follow. 

In order to avoid this manual commanding, they request UMA-MR to implement the behavior of a robot that has to explore (visit) by its own an area as large as possible within an unknown space and in a given time. In addition to map building, an algorithm like this has many other applications: rescue tasks, security works, etc.

The developed algorithm shall be part of a ROS C++/python node. It will be tested employing the Stage simulator previously presented.

For that, your colleagues at UMA-MR have developed the starting code shown below. There are two options, C++ or Python. 

In [ ]:
#include <ros/ros.h>
#include <geometry_msgs/Twist.h>
#include <sensor_msgs/LaserScan.h>
#include <stdlib.h>

int main(int argc, char **argv)
{
  // Initialize the ROS system and become a node.
  ros::init( argc , argv , "exploring") ;

  ROS_INFO_STREAM("Robotic explorer node running and initialized! Let's have some fun!");

  ros::NodeHandle nh;

  // Create a subscriber object
  ros::Subscriber sub = nh.subscribe("/base_scan", 1000, processScanCallback);

  // Create a publisher object
  ros::Publisher pub = nh.advertise <geometry_msgs::Twist>("/cmd_vel" , 1000);

  // Seed the random number generator.
  srand ( time (0) ) ;

  // Loop at 2Hz until the node is shut down or during 5 minutes
  ros::Rate rate (2) ;
  ros::Time begin = ros::Time::now();

  while ( begin.toSec() == 0 )
    begin = ros::Time::now();

  double ellapsed_time = 0;

  while ( ( ros::ok () ) && ( ellapsed_time < 60*5 ) )
  {
    // Create and fill in the message. The other four
    // fields , which are ignored by stage, default to 0.

    geometry_msgs::Twist msg;
    msg.linear.x = 2*double( rand() ) / double(RAND_MAX);
    msg.angular.z = ( 4*double(rand()) / double(RAND_MAX) ) - 2;

    // Publish the message.

    pub.publish(msg) ;

    // Send a message to rosout with the details .
    ROS_INFO_STREAM("Sending random velocity command: "
	    << " linear=" << msg.linear.x
	    << " angular=" << msg.angular.z ) ;

	// Wait until it 's time for another iteration .
    rate.sleep ();
    ros::spinOnce(); // give time to receive /base_scan messages

    ros::Time current = ros::Time::now();
    ellapsed_time = (current - begin).toSec();
    ROS_INFO_STREAM("Ellpased time: " << ellapsed_time );
  }
}


void processScanCallback(const sensor_msgs::LaserScan::ConstPtr& msg)
{
  int n_ranges = msg->ranges.size();
  double nearest = *(std::min_element(msg->ranges.begin(),msg->ranges.end()));
  ROS_INFO_STREAM("I've a total of " << n_ranges << " measurements to process! Are you ready? Nearest=" << nearest );
}



In [ ]:
#!/usr/bin/env python2

import time

import rospy as ros
import numpy as np
from numpy import random

from geometry_msgs.msg import Twist 
from sensor_msgs.msg import LaserScan

def processScanCallback(msg):
    n_ranges = len(msg.ranges)
    nearest = min(msg.ranges)
    ros.loginfo("I've a total of {} measurements to process! Are you ready? Nearest = {}".format(n_ranges, nearest))

def main():
    ros.init_node('exploring')
    ros.loginfo("Robotic explorer node running and initialized! Let's have some fun!")

    scan_sub = ros.Subscriber("/base_scan", LaserScan, processScanCallback, buff_size=1000)
    move_pub = ros.Publisher("/cmd_vel", Twist, queue_size=1000)

    random.seed(time.time())

    rate = ros.Rate(2)
    begin = ros.Time.now()

    while begin.to_sec() == 0:
        begin = ros.Time.now()

    ellapsed_time = 0

    while not ros.is_shutdown() and ellapsed_time < 60.0*5.0:

        msg = Twist()
        msg.linear.x = 2*random.rand()
        msg.angular.z = 4*random.rand()-2

        move_pub.publish(msg)

        ros.loginfo("[robot_nav] Pub twist [{}, {}] ".format(msg.linear.x, msg.angular.z))

        rate.sleep()

        current = ros.Time().now()
        ellapsed_time = current.to_sec() - begin.to_sec()
        ros.loginfo('[robot_explorer] Ellpased time: {}'.format(ellapsed_time))


if __name__ == "__main__":
    main()

### **<span style="color:green"><b><i>ASSIGNMENT 3a: Developing the robotic explorer algorithm</i></b></span>** 

Now it’s time to develop the algorithm for the robot to explore unknown environments! You can extend the C++ code above, located at: `/home/robotics/catkin_ws/src/robotic_explorer/src/robotic_explorer_node.cpp`, or the Python code provided in the course GitHub repository, following the next indications:

- You are allowed to modify the code, not only extend it. The only thing that you have to keep is the execution time limit of 5 minutes, i.e., the robot has 5 minutes to explore/visit as many rooms as possible.
- You can get inspiration from the internet. The efficient exploration of an unknown environment by a mobile robot has been widely studied in the literature, so there are many works addressing it. If you do it, include the inspirational works into the bibliography of the exercise’s deliverable.
- Hint: you can subscribe to any topic published by the `stageros` node, with the exception of `base_pose_ground_truth`.

The code can be edited with your favorite text editor or IDE. There is information on the internet about how to work with different IDEs to develop a ROS package/node. Example of commonly used IDES are QtCreator or KDevelop for C++, or PyCharm or Spyder for Python.

**What to provide?** 

Include your developed code in the code cell below, and describe how it works in the following lines:

<p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>

In [ ]:
# Your developed code for the robotic explorer here

### **<span style="color:green"><b><i>ASSIGNMENT 3b: Testing your algorithm</i></b></span>** 

There are 5 different environments to test your robotic explorer. You can launch them with the following commands (see bellow), or by reusing the `roslaunch` files provided: 

```
rosrun stage_ros stageros $(rospack find missions_pkg)/world/robotics-house1.world
rosrun stage_ros stageros $(rospack find missions_pkg)/world/robotics-house2.world
rosrun stage_ros stageros $(rospack find missions_pkg)/world/robotics-house3.world
rosrun stage_ros stageros $(rospack find missions_pkg)/world/robotics-house4.world
rosrun stage_ros stageros $(rospack find missions_pkg)/world/robotics-offices1.world
```

These images illustrates some of these environments: $\\[10px]$

<center>
<table>
<tr><td>
<img src="./images/house_example_1.png"/>
</td><td>
<img src="./images/house_example_2.png"/>
</td></tr>
<tr><td>
<img src="./images/house_example_3.png"/>
</td><td>
<img src="./images/house_example_4.png"/>
</td></tr>
</table>
<figcaption>Fig 5. Examples of different environments to explore.</figcaption>
</center>

**What to provide?**

Execute your `robotic_explorer_node` for each of the 5 provided environments and count the number of rooms visited by the robot in each of them. Include the initial room as a visited one, and if a room is visited twice or more, then count it only once. Include: 

- Images showing the path followed by the robot in each environment.
- The number of rooms visited by the robot in each environment.
    
  <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>
